<a href="https://colab.research.google.com/github/DinaZakaria/AFLW2000_HeadPose/blob/master/01_ML1_Project_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
import PIL.Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from skimage.io import imread, imsave
from glob import glob
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Functions

In [ ]:
def get_image_XY (image, mat_file):
  XY_crop = []
  mat = sio.loadmat(mat_file)
  XY = mat['pt3d_68'][0:2]
  X_max = XY[0].max()
  Y_max = XY[1].max()
  X_min = XY[0].min()
  Y_min = XY[1].min()
  image = imread (image)
  croped =image[int(Y_min):int(Y_max), int(X_min):int(X_max)]
  X_crop = XY[0]-X_min
  Y_crop = XY[1]-Y_min
  XY_crop.append(X_crop)
  XY_crop.append(Y_crop)
  XY_crop = np.array(XY_crop).flatten()
  return XY_crop

In [ ]:
def get_pyr(mat_file):
  #pitch, yaw, roll
  pyr = mat_file['Pose_Para'][:, 0:3][0]
  return pyr

In [ ]:
def create_features(image_path_list, mat_path_list):
  features = list()
  for i in range (0,len(image_path_list)):
    XY = get_image_XY(image_path_list[i], mat_path_list[i])
    #print(XY)
    features.append(XY)
  return np.array(features)

In [ ]:
def create_labels (mat_path_list):
  labels = list()
  for i in  range(0, len(mat_path_list)):
    mat_file = sio.loadmat(mat_path_list[i])
    pyr = get_pyr(mat_file)
    labels.append(pyr)
  return np.array(labels)

In [ ]:
def create_df(image_path_list, mat_path_list):
  # data = np.array([])
  features = create_features(image_path_list, mat_path_list)
  labels = create_labels(mat_path_list)
  data = np.concatenate((features, labels), axis = 1)
  df = pd.DataFrame(data)
  return df

# Reading Dataset

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Path = 'drive/My Drive/AI-Pro Track ITI/ML1/Project/DATA/aflw2000_3d_data'


In [ ]:
image_path_list= []
image_path_list.extend(glob(os.path.join(Path, "AFLW2000", "*.jpg")))
mat_path_list = []
mat_path_list.extend(glob(os.path.join(Path, "AFLW2000", "*.mat")))

# for i, image_path in enumerate(image_path_list):
#     # read image
#     image = imread(image_path)
print(len(image_path_list))
print(len(mat_path_list))


2000
2000


# Dataframe

In [ ]:
df = create_df(image_path_list, mat_path_list)
df

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,0.000000,0.981430,3.177963,7.520325,16.636810,36.479370,62.062347,88.868835,110.733765,118.650955,...,102.307434,104.603790,105.392700,106.522430,105.473969,104.190979,102.097076,-0.068547,-0.854021,0.218333
1,0.000000,2.627716,8.086754,14.034782,24.569130,43.002541,65.318535,94.591743,129.564789,159.050568,...,124.945862,123.593628,123.707397,127.578278,132.781006,133.528320,133.139709,0.027256,-0.247779,-0.010263
2,0.000000,2.811073,8.599014,14.774353,24.622337,42.561806,64.624336,88.097191,116.038246,139.069031,...,124.044159,123.918488,124.202271,127.209229,130.290009,131.033295,129.797485,-0.167856,-0.361028,0.058591
3,9.189560,13.021408,19.371765,24.665512,31.373764,41.456253,50.984695,61.400986,85.168381,112.612656,...,129.959900,131.272278,131.317719,133.843475,145.569031,147.044464,143.419983,-0.373085,0.293578,-0.125011
4,0.000000,5.628967,13.232834,21.585968,34.941544,56.668945,82.074783,110.237274,139.773102,160.022217,...,110.728516,109.649475,109.133148,110.911469,111.923309,113.186951,113.483612,0.036318,-0.447323,-0.115917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,21.561691,25.811447,32.422134,38.263092,43.471085,47.248810,47.666733,52.505356,69.010727,92.375259,...,101.394165,101.891266,101.157593,99.480896,106.234344,107.861328,106.594971,-0.218161,0.493541,-0.201171
1996,65.364685,68.758057,73.024994,74.687347,73.470276,67.888962,58.108994,49.526123,55.484039,73.191803,...,104.679474,99.255981,94.589630,91.891541,97.009796,101.801483,106.304535,0.236384,0.983607,-0.383734
1997,4.427643,9.238068,17.643219,24.182190,29.904846,36.717194,43.515259,57.460648,85.145340,116.014679,...,121.408234,120.166931,119.589172,121.766174,126.348846,127.412476,128.160187,0.004142,0.229272,-0.080178
1998,22.984161,36.283844,51.948059,65.758560,79.150574,91.497543,101.220642,112.787872,133.944183,155.966736,...,130.242004,125.732666,121.596771,112.713654,119.805725,124.505981,128.966248,-0.075998,0.314266,-0.553940


In [ ]:
df.to_csv("AFLW200_CSV.csv")